In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
chars = ""
with open("vocab.txt", 'r', encoding='utf-8') as f:
        txt = f.read()
        chars = sorted(list(set(txt)))
        
vocab_size = len(chars)

In [3]:
vocab_size

32172

In [4]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l : ''.join([int_to_string[i] for i in l])

In [5]:
import helper as hlp

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
device

'cuda'

In [8]:
block_size = 64

In [9]:
batch_size = 32

In [10]:
n_emb = 384

In [11]:
n_layer = 4

In [12]:
n_head = 4

In [13]:
dropout = 0.2

In [14]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_emb, head_size, bias=False)
        self.query = nn.Linear(n_emb, head_size, bias=False)
        self.value = nn.Linear(n_emb, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [15]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_emb),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_emb),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [16]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_emb)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out

In [17]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_emb, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_emb // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_emb)
        self.ln1 = nn.LayerNorm(n_emb)
        self.ln2 = nn.LayerNorm(n_emb)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

In [18]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_embedding_table = nn.Embedding(block_size, n_emb)
        self.blocks = nn.Sequential(*[Block(n_emb, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_emb) # final layer norm
        self.lm_head = nn.Linear(n_emb, vocab_size)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

In [23]:
model = GPTLanguageModel(len(chars)).to(device)

In [21]:
import pickle

In [22]:
with open('model-01.pkl','rb') as f:
    model = pickle.load(f)

In [23]:
optimizer = torch.optim.AdamW(params=model.parameters(),lr=0.0001)

In [28]:
eval_iters = 100

In [40]:
def estimate_accuracy():
    correct_tokens = 0
    total_tokens = 0

    model.eval()
    with torch.no_grad():
        for split in ['train', 'val']:
            for _ in range(eval_iters):
                X, Y = hlp.get_batch(split=split, batch_size=batch_size, block_size=block_size, encode=encode)
                logits, loss = model(X, Y)
                
                # Get the index of the max log-probability along the last dimension
                predictions = logits.argmax(dim=-1).view(-1)  # Reshape predictions to a 1D tensor
                Y_flat = Y.view(-1)  # Reshape ground truth to a 1D tensor

                correct_tokens += (predictions == Y_flat).sum().item()
                total_tokens += Y_flat.numel()

    model.train()

    accuracy = correct_tokens / total_tokens * 100
    return accuracy


In [48]:
for iter in range(5000):
    if iter % eval_iters == 0:
        accuracy = estimate_accuracy()
        print(f"step: {iter}, accuracy: {accuracy:.2f}%")

    # sample a batch of data
    xb, yb = hlp.get_batch(split='train',batch_size=batch_size,block_size=block_size,encode=encode)

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, accuracy: 23.28%
step: 100, accuracy: 25.17%
step: 200, accuracy: 26.71%
step: 300, accuracy: 24.87%
step: 400, accuracy: 26.00%
step: 500, accuracy: 23.23%
step: 600, accuracy: 25.13%
step: 700, accuracy: 26.50%
step: 800, accuracy: 25.81%
step: 900, accuracy: 26.14%
step: 1000, accuracy: 26.68%
step: 1100, accuracy: 26.15%
step: 1200, accuracy: 26.08%
step: 1300, accuracy: 25.36%
step: 1400, accuracy: 26.41%
step: 1500, accuracy: 25.25%
step: 1600, accuracy: 24.39%
step: 1700, accuracy: 24.02%
step: 1800, accuracy: 27.21%
step: 1900, accuracy: 26.71%
step: 2000, accuracy: 24.57%
step: 2100, accuracy: 25.65%
step: 2200, accuracy: 25.97%
step: 2300, accuracy: 24.80%
step: 2400, accuracy: 24.64%
step: 2500, accuracy: 25.55%
step: 2600, accuracy: 27.08%
step: 2700, accuracy: 26.88%
step: 2800, accuracy: 27.71%
step: 2900, accuracy: 25.64%
step: 3000, accuracy: 25.57%
step: 3100, accuracy: 26.55%
step: 3200, accuracy: 26.76%
step: 3300, accuracy: 26.63%
step: 3400, accuracy: 25.2

In [ ]:
2.801658868789673

In [19]:
import pickle

In [44]:
with open('model-01.pkl', 'wb') as f:
    pickle.dump(model,f)
    print("Done")

Done


In [46]:
prompt = input("Prompt:\n")
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(f'Completion:\n{generated_chars}')

Completion:
hi a dlici snhw..eofepnoo nsigpe uraof re o 2ea.  ildisrede1eei  ,en  iob ieot ethfh  t:n n2eiwglp lue
